# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Develop-Thermodynamic-kinetic-Maximum-Entropy-Model" data-toc-modified-id="Develop-Thermodynamic-kinetic-Maximum-Entropy-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Develop Thermodynamic-kinetic Maximum Entropy Model</a></div><div class="lev2 toc-item"><a href="#Reactions-File" data-toc-modified-id="Reactions-File-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Reactions File</a></div><div class="lev3 toc-item"><a href="#Read-the-file-into-a-dataframe-and-create-a-stoichiometric-matrix" data-toc-modified-id="Read-the-file-into-a-dataframe-and-create-a-stoichiometric-matrix-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Read the file into a dataframe and create a stoichiometric matrix</a></div><div class="lev2 toc-item"><a href="#Calculate-Standard-Free-Energies-of-Reaction" data-toc-modified-id="Calculate-Standard-Free-Energies-of-Reaction-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calculate Standard Free Energies of Reaction</a></div><div class="lev3 toc-item"><a href="#Determine-Pyruvate-transport-reaction-manually" data-toc-modified-id="Determine-Pyruvate-transport-reaction-manually-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Determine Pyruvate transport reaction manually</a></div><div class="lev3 toc-item"><a href="#Output-the-Standard-Reaction-Free-Energies-for-use-in-a-Boltzmann-Simulation" data-toc-modified-id="Output-the-Standard-Reaction-Free-Energies-for-use-in-a-Boltzmann-Simulation-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Output the Standard Reaction Free Energies for use in a Boltzmann Simulation</a></div><div class="lev2 toc-item"><a href="#Set-Fixed-Concentrations/Boundary-Conditions" data-toc-modified-id="Set-Fixed-Concentrations/Boundary-Conditions-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Set Fixed Concentrations/Boundary Conditions</a></div><div class="lev2 toc-item"><a href="#Prepare-model-for-optimization" data-toc-modified-id="Prepare-model-for-optimization-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Prepare model for optimization</a></div><div class="lev1 toc-item"><a href="#Nonlinear-Least-Squares-Optimization-of-Concentrations" data-toc-modified-id="Nonlinear-Least-Squares-Optimization-of-Concentrations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Nonlinear Least Squares Optimization of Concentrations</a></div><div class="lev2 toc-item"><a href="#Apply-Regulation-and-Optimize" data-toc-modified-id="Apply-Regulation-and-Optimize-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Apply Regulation and Optimize</a></div><div class="lev1 toc-item"><a href="#ODE-Simulations" data-toc-modified-id="ODE-Simulations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>ODE Simulations</a></div><div class="lev2 toc-item"><a href="#Calculate-Rate-Constants" data-toc-modified-id="Calculate-Rate-Constants-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Calculate Rate Constants</a></div><div class="lev2 toc-item"><a href="#ODE-Solvers:-Python-interface-using-libroadrunner-to-Sundials/CVODE" data-toc-modified-id="ODE-Solvers:-Python-interface-using-libroadrunner-to-Sundials/CVODE-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>ODE Solvers: Python interface using libroadrunner to Sundials/CVODE</a></div>

# Develop Thermodynamic-kinetic Maximum Entropy Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import re
import os

from IPython.core.display import display

from IPython.core.debugger import set_trace
pd.set_option('display.max_columns', None,'display.max_rows', None)

%matplotlib inline
T = 298.15
R = 8.314e-03
RT = R*T
N_avogadro = 6.022140857e+23
VolCell = 1.0e-15
Concentration2Count = N_avogadro * VolCell
concentration_increment = 1/(N_avogadro*VolCell)

use_experimental_data=True

In [ ]:
cwd = os.getcwd()
print (cwd)

## Reactions File

In [3]:
with open( cwd + '/TCA_PPP_Glycolysis.dat', 'r') as f:
    print(f.read())    

REACTION	CSm
LEFT	OXALOACETATE:MITOCHONDRIA + ACETYL-COA:MITOCHONDRIA + H2O:MITOCHONDRIA
RIGHT	CITRATE:MITOCHONDRIA + COA:MITOCHONDRIA
LEFT_COMPARTMENT
RIGHT_COMPARTMENT
ENZYME_LEVEL	1.0
\\
REACTION	ACONTm
LEFT	CITRATE:MITOCHONDRIA
RIGHT	ISOCITRATE:MITOCHONDRIA
LEFT_COMPARTMENT
RIGHT_COMPARTMENT
ENZYME_LEVEL	1.0
\\
REACTION	ICDHxm
LEFT	ISOCITRATE:MITOCHONDRIA + NAD+:MITOCHONDRIA
RIGHT	2-OXOGLUTARATE:MITOCHONDRIA + NADH:MITOCHONDRIA + CO2:MITOCHONDRIA
LEFT_COMPARTMENT
RIGHT_COMPARTMENT
ENZYME_LEVEL	1.0
\\
REACTION	AKGDam
LEFT	2-OXOGLUTARATE:MITOCHONDRIA + COA:MITOCHONDRIA + NAD+:MITOCHONDRIA
RIGHT	SUCCINYL-COA:MITOCHONDRIA + CO2:MITOCHONDRIA + NADH:MITOCHONDRIA
LEFT_COMPARTMENT
RIGHT_COMPARTMENT
ENZYME_LEVEL	1.0
\\
REACTION	SUCOASm
LEFT	SUCCINYL-COA:MITOCHONDRIA + ADP:MITOCHONDRIA + ORTHOPHOSPHATE:MITOCHONDRIA
RIGHT	SUCCINATE:MITOCHONDRIA + ATP:MITOCHONDRIA + COA:MITOCHONDRIA
LEFT_COMPARTMENT
RIGHT_COMPARTMENT
ENZYME_LEVEL	1.0
\\
REACTION	SUCD1m
LEFT	SUCCINATE:MITOCHONDRIA + NAD+:MITOCH

### Read the file into a dataframe and create a stoichiometric matrix

In [4]:
fdat = open('TCA_PPP_Glycolysis.dat', 'r')

left ='LEFT'
right = 'RIGHT'
left_compartment = 'LEFT_COMPARTMENT'
right_compartment = 'RIGHT_COMPARTMENT'
enzyme_level = 'ENZYME_LEVEL'
deltag0 = 'DGZERO'
deltag0_sigma = 'DGZERO StdDev'
same_compartment = 'Same Compartment?'
full_rxn = 'Full Rxn'

reactions = pd.DataFrame(index=[],columns=[left, right, left_compartment, right_compartment, enzyme_level, deltag0, deltag0_sigma, same_compartment,full_rxn])
reactions.index.name='REACTION'
S_matrix = pd.DataFrame(index=[],columns=[enzyme_level])
S_matrix.index.name='REACTION'

for line in fdat:
    if (line.startswith('REACTION')):
        rxn_name = line[9:-1].lstrip()
        S_matrix.loc[rxn_name,enzyme_level] = 1.0
        reactions.loc[rxn_name,enzyme_level] = 1.0
    if (re.match("^LEFT\s",line)):
        line = line.upper()
        left_rxn = line[4:-1].lstrip()
        left_rxn = re.sub(r'\s+$', '', left_rxn) #Remove trailing white space
        reactions.loc[rxn_name,left] = left_rxn
    elif (re.match('^RIGHT\s',line)):
        line = line.upper()
        right_rxn = line[5:-1].lstrip()
        right_rxn = re.sub(r'\s+$', '', right_rxn) #Remove trailing white space
        reactions.loc[rxn_name,right] = right_rxn    
    elif (line.startswith(left_compartment)):
        cpt_name = line[16:-1].lstrip()
        reactions.loc[rxn_name,left_compartment] = cpt_name
        reactants = re.split(' \+ ',left_rxn)
        for idx in reactants:
            values = re.split(' ', idx);
            if len(values) == 2:
                stoichiometry = np.float64(values[0]);
                molecule = values[1];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            else:
                stoichiometry = np.float64(-1.0);
                molecule = values[0]; 
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            S_matrix.loc[rxn_name,molecule] = stoichiometry;
    elif (line.startswith(right_compartment)):
        cpt_name = line[17:-1].lstrip()
        reactions.loc[rxn_name,right_compartment] = cpt_name
        products = re.split(' \+ ',right_rxn)
        for idx in products:
            values = re.split(' ', idx);
            if len(values) == 2:
                stoichiometry = np.float64(values[0]);
                molecule = values[1];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            else:
                stoichiometry = np.float64(1.0);
                molecule = values[0];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            S_matrix.loc[rxn_name,molecule] = stoichiometry;
    elif (re.match("^ENZYME_LEVEL\s", line)):
        level = line[12:-1].lstrip()
        reactions.loc[rxn_name,enzyme_level] = float(level)
        S_matrix.loc[rxn_name,enzyme_level] = float(level)       
    elif re.match('^COMMENT',line):
        continue
    elif re.match(r'//',line):
        continue
    elif re.match('^#',line):
        continue
        
fdat.close()
S_matrix.fillna(0,inplace=True)
S_active = S_matrix[S_matrix[enzyme_level] > 0.0]
active_reactions = reactions[reactions[enzyme_level] > 0.0]
del S_active[enzyme_level]

# Delete any columns/metabolites that have all zeros in the S matrix:
S_active = S_active.loc[:, (S_active != 0).any(axis=0)]
np.shape(S_active.values)
display(S_active.shape)
display(S_active)
reactions[full_rxn] = reactions[left] + ' = ' + reactions[right]

(29, 47)

,OXALOACETATE:MITOCHONDRIA,ACETYL-COA:MITOCHONDRIA,H2O:MITOCHONDRIA,CITRATE:MITOCHONDRIA,COA:MITOCHONDRIA,ISOCITRATE:MITOCHONDRIA,NAD+:MITOCHONDRIA,2-OXOGLUTARATE:MITOCHONDRIA,NADH:MITOCHONDRIA,CO2:MITOCHONDRIA,SUCCINYL-COA:MITOCHONDRIA,ADP:MITOCHONDRIA,ORTHOPHOSPHATE:MITOCHONDRIA,SUCCINATE:MITOCHONDRIA,ATP:MITOCHONDRIA,FUMARATE:MITOCHONDRIA,(S)-MALATE:MITOCHONDRIA,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,ORTHOPHOSPHATE:CYTOSOL,NAD+:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL,NADH:CYTOSOL,ADP:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,ATP:CYTOSOL,GLYCERONE_PHOSPHATE:CYTOSOL,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",PHOSPHOENOLPYRUVATE:CYTOSOL,PYRUVATE:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,H2O:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,PYRUVATE:MITOCHONDRIA,NADP+:CYTOSOL,"D-GLUCONO-1,5-LACTONE_6-PHOSPHATE:CYTOSOL",NADPH:CYTOSOL,6-PHOSPHO-D-GLUCONATE:CYTOSOL,D-RIBULOSE-5-PHOSPHATE:CYTOSOL,CO2:CYTOSOL,D-XYLULOSE-5-PHOSPHATE:CYTOSOL,D-RIBOSE-5-PHOSPHATE:CYTOSOL,SEDOHEPTULOSE_7-PHOSPHATE:CYTOSOL,D-ERYTHROSE-4-PHOSPHATE:CYTOSOL,L-GLUTAMATE:MITOCHONDRIA,L-GLUTAMINE:MITOCHONDRIA
REACTION,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CSm,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACONTm,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ICDHxm,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AKGDam,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SUCOASm,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SUCD1m,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FUMm,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MDHm,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GAPD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
for idx in reactions.index:
    boltzmann_rxn_str = reactions.loc[idx,'Full Rxn']
    if re.search(':',boltzmann_rxn_str):
        all_cmprts = re.findall(':\S+', boltzmann_rxn_str)
        [s.replace(':', '') for s in all_cmprts] # remove all the ':'s 
        different_compartments = 0
        for cmpt in all_cmprts:
            if not re.match(all_cmprts[0],cmpt):
                different_compartments = 1
        if ((not different_compartments) and (reactions[left_compartment].isnull or reactions[right_compartment].isnull)):
            reactions.loc[idx,left_compartment] = cmpt
            reactions.loc[idx,right_compartment] = cmpt
            reactions.loc[idx,same_compartment] = True
        if different_compartments:
            reactions.loc[idx,same_compartment] = False
    else:
        if (reactions.loc[idx,left_compartment] == reactions.loc[idx,right_compartment]):
            reactions.loc[idx,same_compartment] = True
        else:
            reactions.loc[idx,same_compartment] = False
display(reactions)                
            

,LEFT,RIGHT,LEFT_COMPARTMENT,RIGHT_COMPARTMENT,ENZYME_LEVEL,DGZERO,DGZERO StdDev,Same Compartment?,Full Rxn
REACTION,,,,,,,,,
CSm,OXALOACETATE:MITOCHONDRIA + ACETYL-COA:MITOCHO...,CITRATE:MITOCHONDRIA + COA:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,NaN,NaN,True,OXALOACETATE:MITOCHONDRIA + ACETYL-COA:MITOCHO...
ACONTm,CITRATE:MITOCHONDRIA,ISOCITRATE:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,NaN,NaN,True,CITRATE:MITOCHONDRIA = ISOCITRATE:MITOCHONDRIA
ICDHxm,ISOCITRATE:MITOCHONDRIA + NAD+:MITOCHONDRIA,2-OXOGLUTARATE:MITOCHONDRIA + NADH:MITOCHONDRI...,:MITOCHONDRIA,:MITOCHONDRIA,1,NaN,NaN,True,ISOCITRATE:MITOCHONDRIA + NAD+:MITOCHONDRIA = ...
AKGDam,2-OXOGLUTARATE:MITOCHONDRIA + COA:MITOCHONDRIA...,SUCCINYL-COA:MITOCHONDRIA + CO2:MITOCHONDRIA +...,:MITOCHONDRIA,:MITOCHONDRIA,1,NaN,NaN,True,2-OXOGLUTARATE:MITOCHONDRIA + COA:MITOCHONDRIA...
SUCOASm,SUCCINYL-COA:MITOCHONDRIA + ADP:MITOCHONDRIA +...,SUCCINATE:MITOCHONDRIA + ATP:MITOCHONDRIA + CO...,:MITOCHONDRIA,:MITOCHONDRIA,1,NaN,NaN,True,SUCCINYL-COA:MITOCHONDRIA + ADP:MITOCHONDRIA +...
SUCD1m,SUCCINATE:MITOCHONDRIA + NAD+:MITOCHONDRIA,FUMARATE:MITOCHONDRIA + NADH:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,NaN,NaN,True,SUCCINATE:MITOCHONDRIA + NAD+:MITOCHONDRIA = F...
FUMm,FUMARATE:MITOCHONDRIA + H2O:MITOCHONDRIA,(S)-MALATE:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,NaN,NaN,True,FUMARATE:MITOCHONDRIA + H2O:MITOCHONDRIA = (S)...
MDHm,(S)-MALATE:MITOCHONDRIA + NAD+:MITOCHONDRIA,OXALOACETATE:MITOCHONDRIA + NADH:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,NaN,NaN,True,(S)-MALATE:MITOCHONDRIA + NAD+:MITOCHONDRIA = ...
GAPD,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...,:CYTOSOL,:CYTOSOL,1,NaN,NaN,True,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...


## Calculate Standard Free Energies of Reaction 

In [7]:
import sys
sys.path.insert(0, cwd+'\\equilibrator-api-v0.1.8\\build\\lib')

from equilibrator_api import *
from equilibrator_api.reaction_matcher import ReactionMatcher
reaction_matcher = ReactionMatcher()

if (1):
    eq_api = ComponentContribution(pH=7.0, ionic_strength=0.25)  # loads data
    for idx in reactions.index:
        print(idx, flush=True)
        boltzmann_rxn_str = reactions.loc[idx,'Full Rxn']
        full_rxn_str_no_cmprt = re.sub(':\S+','', boltzmann_rxn_str)
        print(full_rxn_str_no_cmprt)
        full_rxn_str_no_cmprt = re.sub('BETA-D-GLUCOSE','D-GLUCOSE',full_rxn_str_no_cmprt )
        rxn = reaction_matcher.match(full_rxn_str_no_cmprt)
        if not rxn.check_full_reaction_balancing():
            print('Reaction %s is not balanced:\n %s\n' % (idx, full_rxn_str_no_cmprt), flush=True)
        dG0_prime, dG0_uncertainty = eq_api.dG0_prime(rxn)
        display(dG0_prime, dG0_uncertainty)
        reactions.loc[idx,deltag0] = dG0_prime
        reactions.loc[idx,deltag0_sigma] = dG0_uncertainty

        reactions.loc['PYRt2m',deltag0] = -RT*np.log(10)
display(reactions)

CSm
OXALOACETATE + ACETYL-COA + H2O = CITRATE + COA


-35.80572968459592

0.9305521760769464

ACONTm
CITRATE = ISOCITRATE


7.629615992963977

...

2.0868208090573925

GOGAT
2 L-GLUTAMATE + NAD+ = 2-OXOGLUTARATE + L-GLUTAMINE + NADH


48.18637576602259

2.0507972360044278

,LEFT,RIGHT,LEFT_COMPARTMENT,RIGHT_COMPARTMENT,ENZYME_LEVEL,DGZERO,DGZERO StdDev,Same Compartment?,Full Rxn
REACTION,,,,,,,,,
CSm,OXALOACETATE:MITOCHONDRIA + ACETYL-COA:MITOCHO...,CITRATE:MITOCHONDRIA + COA:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,-35.8057,0.930552,True,OXALOACETATE:MITOCHONDRIA + ACETYL-COA:MITOCHO...
ACONTm,CITRATE:MITOCHONDRIA,ISOCITRATE:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,7.62962,0.733847,True,CITRATE:MITOCHONDRIA = ISOCITRATE:MITOCHONDRIA
ICDHxm,ISOCITRATE:MITOCHONDRIA + NAD+:MITOCHONDRIA,2-OXOGLUTARATE:MITOCHONDRIA + NADH:MITOCHONDRI...,:MITOCHONDRIA,:MITOCHONDRIA,1,-2.6492,7.62095,True,ISOCITRATE:MITOCHONDRIA + NAD+:MITOCHONDRIA = ...
AKGDam,2-OXOGLUTARATE:MITOCHONDRIA + COA:MITOCHONDRIA...,SUCCINYL-COA:MITOCHONDRIA + CO2:MITOCHONDRIA +...,:MITOCHONDRIA,:MITOCHONDRIA,1,-37.245,7.97121,True,2-OXOGLUTARATE:MITOCHONDRIA + COA:MITOCHONDRIA...
SUCOASm,SUCCINYL-COA:MITOCHONDRIA + ADP:MITOCHONDRIA +...,SUCCINATE:MITOCHONDRIA + ATP:MITOCHONDRIA + CO...,:MITOCHONDRIA,:MITOCHONDRIA,1,2.01842,1.48197,True,SUCCINYL-COA:MITOCHONDRIA + ADP:MITOCHONDRIA +...
SUCD1m,SUCCINATE:MITOCHONDRIA + NAD+:MITOCHONDRIA,FUMARATE:MITOCHONDRIA + NADH:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,62.364,2.31948,True,SUCCINATE:MITOCHONDRIA + NAD+:MITOCHONDRIA = F...
FUMm,FUMARATE:MITOCHONDRIA + H2O:MITOCHONDRIA,(S)-MALATE:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,-3.44728,0.607693,True,FUMARATE:MITOCHONDRIA + H2O:MITOCHONDRIA = (S)...
MDHm,(S)-MALATE:MITOCHONDRIA + NAD+:MITOCHONDRIA,OXALOACETATE:MITOCHONDRIA + NADH:MITOCHONDRIA,:MITOCHONDRIA,:MITOCHONDRIA,1,29.5419,0.422376,True,(S)-MALATE:MITOCHONDRIA + NAD+:MITOCHONDRIA = ...
GAPD,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...,:CYTOSOL,:CYTOSOL,1,5.24202,0.895659,True,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...


### Determine Pyruvate transport reaction manually

In [8]:
reactions.loc['PYRt2m',deltag0] = -RT*np.log(10)
reactions.loc['PYRt2m',deltag0_sigma] = 0
reactions.loc['SUCD1m',deltag0] = 0

### Output the Standard Reaction Free Energies for use in a Boltzmann Simulation

In [9]:

#reaction_file = open('neurospora_aerobic_respiration.keq', 'w')
reaction_file = open('TCA_PPP_Glycolysis.keq', 'w')
for y in reactions.index:
    print('%s\t%e' % (y, np.exp(-reactions.loc[y,'DGZERO']/RT)),file=reaction_file)
reaction_file.close()    

#reaction_file = open('neurospora_aerobic_respiration.equilibrator.dat', 'w')
reaction_file = open('matlab_reaction_TCA2.dat', 'w')
for y in reactions.index:
    print("REACTION\t",y,file=reaction_file)
    #print(reaction_df[y])
    for x in reactions.columns:
        if x == "Full Rxn":
            continue
        if x == same_compartment:
            continue
#        if ((x == deltag0) and (reactions.loc[y,same_compartment] == False)):
#            continue
        if pd.notnull(reactions.loc[y,x]):
            print(x, reactions.loc[y,x],file=reaction_file)
    print("DGZERO-UNITS    KJ/MOL",file=reaction_file)
    print("//",file=reaction_file)
reaction_file.close()    
display((S_active.columns))

Index(['OXALOACETATE:MITOCHONDRIA', 'ACETYL-COA:MITOCHONDRIA',
       'H2O:MITOCHONDRIA', 'CITRATE:MITOCHONDRIA', 'COA:MITOCHONDRIA',
       'ISOCITRATE:MITOCHONDRIA', 'NAD+:MITOCHONDRIA',
       '2-OXOGLUTARATE:MITOCHONDRIA', 'NADH:MITOCHONDRIA', 'CO2:MITOCHONDRIA',
       'SUCCINYL-COA:MITOCHONDRIA', 'ADP:MITOCHONDRIA',
       'ORTHOPHOSPHATE:MITOCHONDRIA', 'SUCCINATE:MITOCHONDRIA',
       'ATP:MITOCHONDRIA', 'FUMARATE:MITOCHONDRIA', '(S)-MALATE:MITOCHONDRIA',
       'D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL', 'ORTHOPHOSPHATE:CYTOSOL',
       'NAD+:CYTOSOL', '3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL',
       'NADH:CYTOSOL', 'ADP:CYTOSOL', '3-PHOSPHO-D-GLYCERATE:CYTOSOL',
       'ATP:CYTOSOL', 'GLYCERONE_PHOSPHATE:CYTOSOL',
       'D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL', 'PHOSPHOENOLPYRUVATE:CYTOSOL',
       'PYRUVATE:CYTOSOL', '2-PHOSPHO-D-GLYCERATE:CYTOSOL', 'H2O:CYTOSOL',
       'BETA-D-GLUCOSE:CYTOSOL', 'BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL',
       'D-FRUCTOSE_6-PHOSPHATE:CYTOSOL', 'PYRUV

## Set Fixed Concentrations/Boundary Conditions

In [11]:
conc = 'Conc'
variable = 'Variable'
conc_exp = 'Conc_Experimental'
metabolites = pd.DataFrame(index = S_active.columns, columns=[conc,conc_exp,variable])
metabolites[conc] = 0.001
metabolites[variable] = True

metabolites.loc['BETA-D-GLUCOSE:CYTOSOL',conc] =	2.000000e-03
metabolites.loc['BETA-D-GLUCOSE:CYTOSOL',variable] =	False
metabolites.loc['NADH:CYTOSOL',conc] =	8.300000e-05
metabolites.loc['NADH:CYTOSOL',variable] =	False
metabolites.loc['NAD+:CYTOSOL',conc] =	2.600000e-03
metabolites.loc['NAD+:CYTOSOL',variable] =	False
metabolites.loc['NADPH:CYTOSOL',conc] = 8.300000e-05
metabolites.loc['NADPH:CYTOSOL',variable] = False
metabolites.loc['NADP+:CYTOSOL',conc] =	2.600000e-03
metabolites.loc['NADP+:CYTOSOL',variable] =	False
metabolites.loc['ATP:CYTOSOL',conc] =	9.600000e-03
metabolites.loc['ATP:CYTOSOL',variable] = False
metabolites.loc['ADP:CYTOSOL',conc] = 5.600000e-04
metabolites.loc['ADP:CYTOSOL',variable] = False
metabolites.loc['ORTHOPHOSPHATE:CYTOSOL',conc] = 2.000000e-02
metabolites.loc['ORTHOPHOSPHATE:CYTOSOL',variable] = False
metabolites.loc['CO2:CYTOSOL',conc] =	1.000000e-04
metabolites.loc['CO2:CYTOSOL',variable] = False
metabolites.loc['H2O:CYTOSOL',conc] =	5.550000e+01
metabolites.loc['H2O:CYTOSOL',variable] = False
metabolites.loc['NADH:MITOCHONDRIA',conc] =	8.300000e-05
metabolites.loc['NADH:MITOCHONDRIA',variable] = False
metabolites.loc['NAD+:MITOCHONDRIA',conc] =	2.600000e-03
metabolites.loc['NAD+:MITOCHONDRIA',variable] =	False
metabolites.loc['ATP:MITOCHONDRIA',conc] =	9.600000e-03
metabolites.loc['ATP:MITOCHONDRIA',variable] = False
metabolites.loc['ADP:MITOCHONDRIA',conc] =	5.600000e-04
metabolites.loc['ADP:MITOCHONDRIA',variable] = False
metabolites.loc['ORTHOPHOSPHATE:MITOCHONDRIA',conc] =	2.000000e-02
metabolites.loc['ORTHOPHOSPHATE:MITOCHONDRIA',variable] = False
metabolites.loc['H2O:MITOCHONDRIA',conc] =	5.550000e+01
metabolites.loc['H2O:MITOCHONDRIA',variable] = False
metabolites.loc['CO2:MITOCHONDRIA',conc] =	1.000000e-04
metabolites.loc['CO2:MITOCHONDRIA',variable] = False
metabolites.loc['COA:MITOCHONDRIA',conc] =	1.400000e-03
metabolites.loc['COA:MITOCHONDRIA',variable] = False
metabolites.loc['L-GLUTAMATE:MITOCHONDRIA',conc] = 9.60e-05
metabolites.loc['L-GLUTAMATE:MITOCHONDRIA',variable] = False 
metabolites.loc['L-GLUTAMINE:MITOCHONDRIA',conc] = 3.81e-03
metabolites.loc['L-GLUTAMINE:MITOCHONDRIA',variable] = False 

#When loading experimental concentrations, first copy current 
#rule of thumb then overwrite with data values.
metabolites[conc_exp] = metabolites[conc]
metabolites.loc['(S)-MALATE:MITOCHONDRIA',conc_exp] = 1.68e-03
metabolites.loc['BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL',conc_exp] = 7.88e-03
metabolites.loc['D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL',conc_exp] = 2.71e-04
metabolites.loc['PYRUVATE:MITOCHONDRIA',conc_exp] = 3.66e-03
metabolites.loc['ISOCITRATE:MITOCHONDRIA',conc_exp] = 1.000000e-03
metabolites.loc['OXALOACETATE:MITOCHONDRIA',conc_exp] = 1.000000e-03
metabolites.loc['3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL',conc_exp] = 1.000000e-03
metabolites.loc['ACETYL-COA:MITOCHONDRIA',conc_exp] = 6.06e-04 
metabolites.loc['CITRATE:MITOCHONDRIA',conc_exp] = 1.96e-03
metabolites.loc['2-OXOGLUTARATE:MITOCHONDRIA',conc_exp] = 4.43e-04
metabolites.loc['FUMARATE:MITOCHONDRIA',conc_exp] = 1.15e-04
metabolites.loc['SUCCINYL-COA:MITOCHONDRIA',conc_exp] = 2.33e-04
metabolites.loc['3-PHOSPHO-D-GLYCERATE:CYTOSOL',conc_exp] = 1.54e-03
metabolites.loc['GLYCERONE_PHOSPHATE:CYTOSOL',conc_exp] = 3.060000e-03
metabolites.loc['SUCCINATE:MITOCHONDRIA',conc_exp] = 5.69e-04
metabolites.loc['D-RIBULOSE-5-PHOSPHATE:CYTOSOL',conc_exp] = 1.12e-04
metabolites.loc['PHOSPHOENOLPYRUVATE:CYTOSOL',conc_exp] = 1.84e-04
metabolites.loc['D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL',conc_exp] = 1.52e-02
metabolites.loc['D-ERYTHROSE-4-PHOSPHATE:CYTOSOL',conc_exp] = 4.90e-05
metabolites.loc['D-XYLULOSE-5-PHOSPHATE:CYTOSOL',conc_exp] = 1.810000e-04
metabolites.loc['D-FRUCTOSE_6-PHOSPHATE:CYTOSOL',conc_exp] = 2.52e-03
metabolites.loc['PYRUVATE:CYTOSOL',conc_exp] = 3.66e-03
metabolites.loc['D-RIBOSE-5-PHOSPHATE:CYTOSOL',conc_exp] = 7.8700e-04
metabolites.loc['SEDOHEPTULOSE_7-PHOSPHATE:CYTOSOL',conc_exp] = 8.82e-04
metabolites.loc['2-PHOSPHO-D-GLYCERATE:CYTOSOL',conc_exp] = 9.180e-05
metabolites.loc['6-PHOSPHO-D-GLUCONATE:CYTOSOL',conc_exp] = 3.77e-03
metabolites.loc['D-GLUCONO-1,5-LACTONE_6-PHOSPHATE:CYTOSOL',conc_exp] = 1.000000e-03

nvariables = metabolites[metabolites[variable]].count()
nvar = nvariables[variable]

metabolites.sort_values(by=variable, axis=0,ascending=False, inplace=True,)
display(metabolites)

,Conc,Conc_Experimental,Variable
OXALOACETATE:MITOCHONDRIA,0.001000,0.001000,True
3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL,0.001000,0.001000,True
GLYCERONE_PHOSPHATE:CYTOSOL,0.001000,0.003060,True
"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",0.001000,0.015200,True
PHOSPHOENOLPYRUVATE:CYTOSOL,0.001000,0.000184,True
PYRUVATE:CYTOSOL,0.001000,0.003660,True
2-PHOSPHO-D-GLYCERATE:CYTOSOL,0.001000,0.000092,True
BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,0.001000,0.007880,True
D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,0.001000,0.002520,True
PYRUVATE:MITOCHONDRIA,0.001000,0.003660,True


## Prepare model for optimization

- Adjust S Matrix to use only reactions with activity > 0, if necessary.
- Water stoichiometry in the stiochiometric matrix needs to be set to zero since water is held constant.
- The initial concentrations of the variable metabolites are random.
- All concentrations are changed to log counts.
- Equilibrium constants are calculated from standard free energies of reaction.
- R (reactant) and P (product) matrices are derived from S.

In [12]:
# Make sure all the indices and columns are in the correct order:
active_reactions = reactions[reactions[enzyme_level] > 0.0]
Sactive_index = S_active.index

active_reactions.reindex(index = Sactive_index, copy = False)
S_active = S_active.reindex(columns = metabolites.index, copy = False)
S_active['H2O:MITOCHONDRIA'] = 0
S_active['H2O:CYTOSOL'] = 0

where_are_NaNs = np.isnan(S_active)
S_active[where_are_NaNs] = 0
S_mat = S_active.values

Keq_constant = np.exp(-active_reactions[deltag0].astype('float')/RT)
Keq_constant = Keq_constant.values

P_mat = np.where(S_mat>0,S_mat,0)
R_back_mat = np.where(S_mat<0, S_mat, 0)
E_regulation = np.ones(Keq_constant.size) # THis is the vector of enzyme activities, Range: 0 to 1.

mu0 = 1 #Dummy parameter for now; reserved for free energies of formation

conc_type=conc
if (use_experimental_data):
    print("USING EXPERIMENTAL DATA")
    conc_type=conc_exp
    
    
variable_concs = np.array(metabolites[conc_type].iloc[0:nvar].values, dtype=np.float64)
v_log_concs = -10 + 10*np.random.rand(nvar) #Vary between 1 M to 1.0e-10 M
v_concs = np.exp(v_log_concs)
v_log_counts_stationary = np.log(v_concs*Concentration2Count)
v_log_counts = v_log_counts_stationary

fixed_concs = np.array(metabolites[conc_type].iloc[nvar:].values, dtype=np.float64)
fixed_counts = fixed_concs*Concentration2Count
f_log_counts = np.log(fixed_counts)

complete_target_log_counts = np.log(Concentration2Count * metabolites[conc_type].values)
target_v_log_counts = complete_target_log_counts[0:nvar]
target_f_log_counts = complete_target_log_counts[nvar:]

delta_increment_for_small_concs = np.zeros(metabolites[conc_type].values.size);

variable_concs_begin = np.array(metabolites[conc_type].iloc[0:nvar].values, dtype=np.float64)

USING EXPERIMENTAL DATA


In [19]:
from scipy.optimize import least_squares
import torch
import max_entropy_functions as me
res_lsq1 = least_squares(me.derivatives, v_log_counts, method='lm',xtol=1e-15, args=(f_log_counts, mu0, S_mat, R_back_mat, P_mat, delta_increment_for_small_concs, Keq_constant, E_regulation))
display(res_lsq1.x)

array([-48.87028054, -10.90651892,   3.44933262,  13.33424796,
       -13.98479593,  -8.34565858, -13.07671205,   5.43690612,
         5.55717305,  -8.59778824,   8.04296413,  14.54196187,
         3.6346578 ,   2.9807751 ,   3.05770363,   6.52967763,
        -0.43259663,  12.73439992,  -8.83622008, -34.01665901,
        -0.33015463, -37.89076355, -37.84226944, -36.67824951,
       -37.56796086, -43.52340199, -52.57074036])

## Apply Regulation and Optimize

In [25]:
#initialize variables
delta_S = np.ones(Keq_constant.size)
delta_S_metab = np.ones(metabolites.size)
E_regulation=np.ones(Keq_constant.size)

v_log_counts = np.log(v_concs*Concentration2Count)

ipolicy=7 #USE 7 or 4
React_Choice=0

v_log_counts = np.log(variable_concs_begin*Concentration2Count)
i=0
while( (i < 500) and (np.max(delta_S_metab) > 0) ):
    reward=0
    
    res_lsq = least_squares(me.derivatives, v_log_counts, method='lm',xtol=1e-15, args=(f_log_counts, mu0, S_mat, R_back_mat, P_mat, delta_increment_for_small_concs, Keq_constant, E_regulation))
    v_log_counts = res_lsq.x
    log_metabolites = np.append(v_log_counts, f_log_counts)
        
    #make calculations to regulate
    rxn_flux = me.oddsDiff(v_log_counts, f_log_counts, mu0, S_mat, R_back_mat, P_mat, delta_increment_for_small_concs, Keq_constant, E_regulation)
        
    KQ_f = me.odds(log_metabolites, mu0,S_mat, R_back_mat, P_mat, delta_increment_for_small_concs,Keq_constant);
    Keq_inverse = np.power(Keq_constant,-1)
    KQ_r = me.odds(log_metabolites, mu0,-S_mat, P_mat, R_back_mat, delta_increment_for_small_concs,Keq_inverse,-1);
    
    epr = me.entropy_production_rate(KQ_f, KQ_r, E_regulation)

    delta_S_metab = me.calc_deltaS_metab(v_log_counts, target_v_log_counts);
    
    delta_S = me.calc_deltaS(v_log_counts,target_v_log_counts, f_log_counts, S_mat, KQ_f);
    
    [RR,Jac] = me.calc_Jac2(v_log_counts, f_log_counts, S_mat, delta_increment_for_small_concs, KQ_f, KQ_r, E_regulation)
    A = me.calc_A(v_log_counts, f_log_counts, S_mat, Jac, E_regulation )
        
    [ccc,fcc] = me.conc_flux_control_coeff(nvar, A, S_mat, rxn_flux, RR)
    
    if (React_Choice == -1) or (np.max(delta_S_metab)<=0):
        
        print("FINISHED OPTIMIZING")
        break
    
    React_Choice = me.get_enzyme2regulate(ipolicy, delta_S_metab, delta_S,
                                        ccc, KQ_f, E_regulation,v_log_counts)
    
        
    newE = me.calc_reg_E_step(E_regulation, React_Choice, nvar, v_log_counts, 
                           f_log_counts, target_v_log_counts, S_mat, A, rxn_flux, KQ_f,
                           delta_S_metab)
        
    E_regulation[React_Choice] = newE
    i += 1

FINISHED OPTIMIZING


# ODE Simulations

## Calculate Rate Constants
Equation 9 from Cannon, et al, Prediction of Metabolite Concentrations, Rate Constants and Post-translational Regulation using Maximum Entropy-based Simulations with Application to Central Metabolism of Neurospora crassa," Processes 6, 63, DOI:10.3390/pr6060063.

In [29]:
def calculate_rate_constants(log_counts, rxn_flux,KQ_inverse, R, E_Regulation):
    KQ = np.power(KQ_inverse,-1)
    #Infer rate constants from reaction flux
    denominator = E_Regulation* np.exp(-R.dot(log_counts))*(1-KQ_inverse)
    # A reaction near equilibrium is problematic because (1-KQ_inverse)->0
    # By setting these reactions to be 
    # rate constant = 1/product_concs we are setting the rate to 1, which
    # is the same as the thermodynammic rate = KQ.
    one_idx, = np.where(KQ_inverse > 0.9)
    denominator[one_idx] = E_Regulation[one_idx]* np.exp(-R[one_idx,:].dot(log_counts));
    rxn_flux[one_idx] = 1;
    fwd_rate_constants = rxn_flux/denominator;
    
    return(fwd_rate_constants)

In [31]:
log_counts = np.append(v_log_counts,f_log_counts)
KQ_inverse = me.odds(log_counts,mu0,S_mat, R_back_mat, P_mat, delta_increment_for_small_concs, Keq_constant, direction = -1)
forward_rate_constants = calculate_rate_constants(log_counts, rxn_flux, KQ_inverse, R_back_mat, E_regulation)
reverse_rate_constants = forward_rate_constants/Keq_constant
display(forward_rate_constants)

array([ 4.93871028e+015,  3.65332876e+017,  1.91064946e+014,
        5.13545873e+010,  1.89700569e+003,  1.83162283e+011,
        8.49496054e+015,  2.22245091e+011,  1.31643161e-001,
        1.61697031e-001,  1.45095104e-001,  7.37238490e-006,
        3.51221235e+000,  8.82164719e+004,  4.77687510e+005,
        1.13380963e-005, -2.05715459e-002,  6.67618352e-010,
        4.21185555e+003,  4.10552582e-009,  2.78008078e-009,
        3.21354999e-004,  3.45545130e-012,  2.63929646e-002,
        2.63929646e-002,  2.38518467e-003,  1.37782276e-002,
        7.82240234e-002,  1.42108539e-314])

## ODE Solvers: Python interface using libroadrunner to Sundials/CVODE